In [1]:
# configure auto-reload and add libs to path
%load_ext autoreload
%autoreload 2

import os, sys
fastai_lib_path = os.path.abspath('../../../../fastai')
if fastai_lib_path not in sys.path: sys.path.append(fastai_lib_path)

In [2]:
# core imports
import json, requests, ast

from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn import metrics

In [3]:
# pandas and plotting config
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

%matplotlib inline

Configure path and TBA Read API key (which is required to use v3 of their API).

In [4]:
PATH = 'data'

comp_yr = 2017

tba_key = 'J9XyDSN69eZMHqanEDaevzAywDjxL9iyBYAQh2erLJJ9MtZVG60HsEYeKVwloFGe'
tba_header = { 'X-TBA-Auth-Key': tba_key }
tba_url = 'https://thebluealliance.com/api/v3'

team_match_scores_path = f'{PATH}/{comp_yr}-team-match-scores-raw.csv'
team_event_stats_path = f'{PATH}/{comp_yr}-team-event-stats-raw.csv'
scouting_reports_path = f'{PATH}/{comp_yr}-scouting-reports-raw.csv'

In [5]:
def replace_dict(df, col, default_val=''):
    new_df = pd.DataFrame((ast.literal_eval(v) for k,v in df[col].iteritems() if (str(v) != 'nan')))
    new_df.columns = [f'{col}_{c}' for c in new_df.columns]
    ret = pd.concat([df,new_df.fillna(default_val)], axis=1)
    
    del ret[col]
    return ret

## 1. Prep the scouting reports

In [6]:
scouting_reports_df = pd.read_csv(scouting_reports_path, low_memory=False)
scouting_reports_df.head()

,asdf,dddd,event_id,rating_overall_gear_efficiency,rating_overall_gear_efficiency_auto,rating_overall_gear_placement_auto,rating_overall_pilot_competency,rating_overall_robot_stability,rating_scoring_airship_climb,rating_scoring_base_line_made_auto,rating_scoring_gears_made,rating_scoring_gears_made_auto,rating_scoring_high_goals_made,rating_scoring_high_goals_made_auto,rating_scoring_low_goals_made,rating_scoring_low_goals_made_auto,scored_at,scored_by,team_id,match_found,match_key
0,NaN,NaN,2017flwp,2.0,NaN,NaN,NaN,2.0,0.0,1.0,3.0,0.0,10.0,0.0,0.0,5.0,1.488653e+12,"{'email': 'wgilliam@ohmeow.com', 'name': 'ohme...",frc125,0,NaN
1,NaN,NaN,2017flwp,1.0,2.0,NaN,NaN,1.0,1.0,1.0,0.0,2.0,20.0,5.0,0.0,15.0,1.488653e+12,"{'email': 'wgilliam@ohmeow.com', 'name': 'ohme...",frc125,0,NaN
2,NaN,NaN,2017flwp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,50.0,0.0,0.0,0.0,1.488678e+12,"{'email': 'wgilliam@ohmeow.com', 'name': 'ohme...",frc263,0,NaN
3,NaN,NaN,2017flwp,5.0,NaN,NaN,NaN,5.0,1.0,1.0,0.0,0.0,30.0,20.0,0.0,0.0,1.488678e+12,"{'email': 'wgilliam@ohmeow.com', 'name': 'ohme...",frc263,0,NaN
4,NaN,NaN,2017casd,3.0,NaN,NaN,NaN,5.0,1.0,NaN,1.0,0.0,0.0,0.0,85.0,10.0,1.489166e+12,"{'email': 'spencer@klawans.me', 'name': 'Spenc...",frc399,1,2017casd_qm1


In [7]:
scouting_reports_df = replace_dict(scouting_reports_df, 'scored_by')

In [8]:
# drop the 'scored_by' column (and other weird/probably test values)
scouting_reports_df.drop(['asdf', 'dddd'], axis=1, inplace=True)

In [9]:
# remove rows without an event_id
scouting_reports_df = scouting_reports_df[pd.notnull(scouting_reports_df['event_id'])]

In [10]:
# convert all NaNs to 0
scouting_reports_df.fillna(0, inplace=True)

In [11]:
scouting_reports_df.rename(columns={'event_id': 'event_key', 'team_id': 'team_key'}, inplace=True)

In [12]:
# convert unix timestamps to datetime
scouting_reports_df['scored_at'] = pd.to_datetime(scouting_reports_df['scored_at'], unit='ms')

# sort by date ascending (most recent on top)
scouting_reports_df.sort_values('scored_at', ascending=False, inplace=True)

In [13]:
d = DataFrameSummary(scouting_reports_df)
# d.columns_stats.transpose()
d.summary().transpose()

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
event_key,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1234,4,0,0%,categorical
rating_overall_gear_efficiency,1234,2.36467,1.64977,0,1,3,4,5,1234,6,0,0%,numeric
rating_overall_gear_efficiency_auto,1234,0.871151,1.61916,0,0,0,1,5,1234,6,0,0%,numeric
rating_overall_gear_placement_auto,1234,0.0794165,0.270497,0,0,0,0,1,1234,2,0,0%,bool
rating_overall_pilot_competency,1234,1.62804,2.07996,0,0,0,4,5,1234,6,0,0%,numeric
rating_overall_robot_stability,1234,3.19773,1.2843,0,3,3,4,5,1234,6,0,0%,numeric
rating_scoring_airship_climb,1234,0.560778,0.496494,0,0,1,1,1,1234,2,0,0%,bool
rating_scoring_base_line_made_auto,1234,0.258509,0.437993,0,0,0,1,1,1234,2,0,0%,bool
rating_scoring_gears_made,1234,1.52917,1.29051,0,0,1,2,6,1234,7,0,0%,numeric
rating_scoring_gears_made_auto,1234,0.197731,0.40651,0,0,0,0,3,1234,4,0,0%,numeric


## 2. Prep the match scores

In [14]:
matches_df = pd.read_csv(team_match_scores_path, low_memory=False)
matches_df.head()

,actual_time,adjustPoints,alliance,alliance_team_keys,autoFuelHigh,autoFuelLow,autoFuelPoints,autoMobilityPoints,autoPoints,autoRotorPoints,comp_level,event_city,event_code,event_country,event_district_display_name,event_district_key,event_end_date,event_first_event_code,event_first_event_id,event_key,event_lat,event_lng,event_name,event_start_date,event_state_prov,event_timezone,event_type,event_type_string,event_week,foulCount,foulPoints,is_winner,kPaBonusPoints,kPaRankingPointAchieved,match_key,match_number,post_result_time,predicted_time,robot1Auto,robot2Auto,robot3Auto,rotor1Auto,rotor1Engaged,rotor2Auto,rotor2Engaged,rotor3Engaged,rotor4Engaged,rotorBonusPoints,rotorRankingPointAchieved,score,set_number,tba_rpEarned,team_key,techFoulCount,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,time,totalPoints,touchpadFar,touchpadMiddle,touchpadNear,winning_margin,year
0,1488551305,0,blue,"['frc3651', 'frc6693']",0,0,0,5,5,0,qm,Myrtle Beach,scmb,USA,NaN,NaN,2017-03-04,SCMB,22531.0,2017scmb,33.70658,-78.874192,Palmetto Regional,2017-03-01,SC,America/New_York,0,Regional,0,0,30,1,0,False,2017scmb_qm4,4,1.488552e+09,1.488552e+09,Mobility,None,None,False,True,False,False,False,False,0,False,75,1,2.0,frc1287,0,0,0,0,40,40,0,1488551040,75,None,None,None,30,2017
1,1488551818,0,red,"['frc3653', 'frc2152']",0,0,0,15,15,0,qm,West Palm Beach,flwp,USA,NaN,NaN,2017-03-04,FLWP,22591.0,2017flwp,26.70457,-80.058875,South Florida Regional,2017-03-01,FL,America/New_York,0,Regional,0,0,0,1,0,False,2017flwp_qm1,1,1.488552e+09,1.488552e+09,Mobility,Mobility,Mobility,False,True,False,False,False,False,0,False,155,1,2.0,frc6038,0,2,0,0,140,40,100,1488551400,155,None,ReadyForTakeoff,ReadyForTakeoff,44,2017
2,1488552377,0,blue,"['frc5196', 'frc6388']",0,0,0,0,0,0,qm,West Palm Beach,flwp,USA,NaN,NaN,2017-03-04,FLWP,22591.0,2017flwp,26.70457,-80.058875,South Florida Regional,2017-03-01,FL,America/New_York,0,Regional,0,0,0,0,0,False,2017flwp_qm2,2,1.488553e+09,1.488553e+09,None,None,None,False,True,False,True,False,False,0,False,80,1,0.0,frc263,0,0,0,0,80,80,0,1488551820,80,None,None,None,-45,2017
3,1488552377,0,red,"['frc694', 'frc2641']",0,0,0,5,5,0,qm,West Palm Beach,flwp,USA,NaN,NaN,2017-03-04,FLWP,22591.0,2017flwp,26.70457,-80.058875,South Florida Regional,2017-03-01,FL,America/New_York,0,Regional,0,0,0,1,0,False,2017flwp_qm2,2,1.488553e+09,1.488553e+09,Mobility,None,None,False,True,False,True,True,False,0,False,125,1,2.0,frc1523,0,0,0,0,120,120,0,1488551820,125,None,None,None,45,2017
4,1488553596,0,blue,"['frc4823', 'frc6366']",0,0,0,10,10,0,qm,Myrtle Beach,scmb,USA,NaN,NaN,2017-03-04,SCMB,22531.0,2017scmb,33.70658,-78.874192,Palmetto Regional,2017-03-01,SC,America/New_York,0,Regional,0,0,25,1,0,False,2017scmb_qm8,8,1.488554e+09,1.488554e+09,None,Mobility,Mobility,False,True,False,True,False,False,0,False,165,1,2.0,frc4451,0,0,0,0,130,80,50,1488552960,165,None,None,ReadyForTakeoff,10,2017


In [15]:
# map autonomous to 1 or 0
print(matches_df.robot1Auto.unique(), matches_df.robot2Auto.unique(), matches_df.robot3Auto.unique())
robotAutoMap = { 'Mobility': 1, 'None': 0, 'Unknown': 0 }

matches_df.robot1Auto = matches_df.robot1Auto.map(robotAutoMap)
matches_df.robot2Auto = matches_df.robot2Auto.map(robotAutoMap)
matches_df.robot3Auto = matches_df.robot3Auto.map(robotAutoMap)

['Mobility' 'None'] ['None' 'Mobility'] ['None' 'Mobility']


In [16]:
# map takeoffs to 1 or 0
print(matches_df.touchpadFar.unique(), matches_df.touchpadMiddle.unique(), matches_df.touchpadNear.unique())
touchpadMap = { 'ReadyForTakeoff': 1, 'None': 0, 'Unknown': 0 }

matches_df.touchpadFar = matches_df.touchpadFar.map(touchpadMap)
matches_df.touchpadMiddle = matches_df.touchpadMiddle.map(touchpadMap)
matches_df.touchpadNear = matches_df.touchpadNear.map(touchpadMap)

['None' 'ReadyForTakeoff'] ['None' 'ReadyForTakeoff'] ['None' 'ReadyForTakeoff']


In [17]:
# convert boolean columns and those that maintain a count to int
int_cols = [
    'rotor1Auto', 'rotor1Engaged', 'rotor2Auto', 
    'rotor2Engaged', 'rotor3Engaged', 'rotor4Engaged',  
]

matches_df[int_cols] = matches_df[int_cols].astype(int)

In [18]:
# create additional columns that aggregate others
matches_df['robotAutoCount'] = matches_df['robot1Auto'] + matches_df['robot2Auto'] + matches_df['robot3Auto']
matches_df['rotorAutoCount'] = matches_df['rotor1Auto'] + matches_df['rotor2Auto']
matches_df['rotorEngagedCount'] = matches_df['rotor1Engaged'] + matches_df['rotor2Engaged'] + matches_df['rotor3Engaged'] + matches_df['rotor4Engaged']
matches_df['takeoffCount'] = matches_df['touchpadFar'] + matches_df['touchpadMiddle'] + matches_df['touchpadNear']

In [19]:
# convert unix timestamps to datetime
datetime_cols = ['actual_time', 'post_result_time', 'predicted_time', 'time']
matches_df[datetime_cols] = matches_df[datetime_cols].apply(pd.to_datetime, unit='s')

# sort data by even start_date (most recent on top)
matches_df.sort_values('actual_time', ascending=False, inplace=True)

In [20]:
# convert NaNs to 0
matches_df.fillna(0.0, inplace=True)

In [21]:
d = DataFrameSummary(matches_df)
# d.columns_stats.transpose()
d.summary().transpose()

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
actual_time,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5417,2756,0,0%,date
adjustPoints,5417,-0.2566,9.82813,-445,0,0,0,0,5417,4,0,0%,numeric
alliance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5417,2,0,0%,bool
alliance_team_keys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5417,4541,0,0%,categorical
autoFuelHigh,5417,3.03397,7.23439,0,0,0,2,56,5417,51,0,0%,numeric
autoFuelLow,5417,0.137161,1.04683,0,0,0,0,10,5417,11,0,0%,numeric
autoFuelPoints,5417,3.07218,7.22559,0,0,0,3,56,5417,51,0,0%,numeric
autoMobilityPoints,5417,13.5352,2.77483,0,15,15,15,15,5417,4,0,0%,numeric
autoPoints,5417,49.105,32.9633,0,15,70,75,166,5417,123,0,0%,numeric
autoRotorPoints,5417,32.4977,30.7966,0,0,60,60,120,5417,3,0,0%,numeric


## 3. Prep event stats

In [22]:
stats_df = pd.read_csv(team_event_stats_path, low_memory=False)
stats_df.head()

,Auto,Match Points,Pressure,Ranking Score,Rotor,Touchpad,ccwms,dprs,event_key,is_finals_team,is_playoff_team,losses,matches_played,oprs,post_qual_wins,qual_average,rank,start_date,team_key,ties,wins
0,339.0,1536.0,6.0,1.50,800.0,550.0,34.356540,44.906465,2017scmb,1,1,2,8,79.263005,5,NaN,6,2017-03-01,frc1287,0,6
1,165.0,1513.0,3.0,1.37,780.0,600.0,31.595175,64.709658,2017scmb,0,1,2,8,96.304833,1,NaN,11,2017-03-01,frc4451,1,5
2,228.0,1453.0,8.0,1.25,760.0,550.0,23.209025,55.120927,2017scmb,1,1,3,8,78.329953,5,NaN,17,2017-03-01,frc283,0,5
3,335.0,1974.0,24.0,1.60,1040.0,700.0,48.955095,48.207602,2017flwp,1,1,2,10,97.162697,5,NaN,3,2017-03-01,frc1523,0,8
4,326.0,1793.0,23.0,1.20,1040.0,550.0,24.413285,52.241866,2017flwp,0,1,4,10,76.655151,2,NaN,15,2017-03-01,frc263,0,6


In [23]:
# convert whatever columns can to be numeric
stats_df = stats_df.apply(pd.to_numeric, errors='ignore')

In [24]:
# convert NaNs to 0
stats_df.fillna(0, inplace=True)

In [25]:
# sort data by even start_date (most recent on top)
stats_df.sort_values('start_date', ascending=False, inplace=True)

In [26]:
d = DataFrameSummary(stats_df)
# d.columns_stats.transpose()
d.summary().transpose()

,count,mean,std,min,25%,50%,75%,max,counts,uniques,missing,missing_perc,types
Auto,437,428.419,180.639,91,292,421,555,988,437,322,0,0%,numeric
Match Points,437,2227.12,639.44,825,1745,2220,2752,3889,437,392,0,0%,numeric
Pressure,437,47.6018,75.0894,0,6,26,58,577,437,126,0,0%,numeric
Ranking Score,437,1.21613,0.447002,0,0.9,1.2,1.5,2.8,437,76,0,0%,numeric
Rotor,437,1115.88,309.29,460,860,1100,1380,1820,437,67,0,0%,numeric
Touchpad,437,854.577,304.499,50,600,900,1100,1650,437,32,0,0%,numeric
ccwms,437,8.70183,35.1521,-99.4342,-14.0945,9.55658,31.5952,111.992,437,437,0,0%,numeric
dprs,437,72.2131,26.442,6.92773,54.5413,72.7121,88.0207,153.803,437,437,0,0%,numeric
event_key,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,437,67,0,0%,categorical
is_finals_team,437,0.19222,0.394496,0,0,0,0,1,437,2,0,0%,bool


## 4. Combine datasets

In [27]:
combined_df = pd.merge(scouting_reports_df, matches_df, how='right', on=['event_key', 'match_key', 'team_key'])

In [28]:
# option 1: merge event stats by event and team
combined_df = pd.merge(combined_df, stats_df, how='inner', on=['event_key', 'team_key'])

# option 2: merge avg. competition stats by team
# stats_by_team_df = stats_df.groupby(by=['team_key']).agg('mean')
# stats_by_team_df.reset_index(inplace=True)
# combined_df = pd.merge(combined_df, stats_by_team_df, how='inner', on=['team_key'])

In [29]:
combined_df.fillna(0.0, inplace=True)
combined_df.sort_values('actual_time', ascending=False, inplace=True)

In [30]:
print(len(combined_df), len(combined_df[combined_df.match_found == 1]))
combined_df[combined_df.match_found == 1].head(20)

5444 1211


,event_key,rating_overall_gear_efficiency,rating_overall_gear_efficiency_auto,rating_overall_gear_placement_auto,rating_overall_pilot_competency,rating_overall_robot_stability,rating_scoring_airship_climb,rating_scoring_base_line_made_auto,rating_scoring_gears_made,rating_scoring_gears_made_auto,rating_scoring_high_goals_made,rating_scoring_high_goals_made_auto,rating_scoring_low_goals_made,rating_scoring_low_goals_made_auto,scored_at,team_key,match_found,match_key,scored_by_email,scored_by_name,scored_by_user_key,actual_time,adjustPoints,alliance,alliance_team_keys,autoFuelHigh,autoFuelLow,autoFuelPoints,autoMobilityPoints,autoPoints,autoRotorPoints,comp_level,event_city,event_code,event_country,event_district_display_name,event_district_key,event_end_date,event_first_event_code,event_first_event_id,event_lat,event_lng,event_name,event_start_date,event_state_prov,event_timezone,event_type,event_type_string,event_week,foulCount,foulPoints,is_winner,kPaBonusPoints,kPaRankingPointAchieved,match_number,post_result_time,predicted_time,robot1Auto,robot2Auto,robot3Auto,rotor1Auto,rotor1Engaged,rotor2Auto,rotor2Engaged,rotor3Engaged,rotor4Engaged,rotorBonusPoints,rotorRankingPointAchieved,score,set_number,tba_rpEarned,techFoulCount,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,time,totalPoints,touchpadFar,touchpadMiddle,touchpadNear,winning_margin,year,robotAutoCount,rotorAutoCount,rotorEngagedCount,takeoffCount,Auto,Match Points,Pressure,Ranking Score,Rotor,Touchpad,ccwms,dprs,is_finals_team,is_playoff_team,losses,matches_played,oprs,post_qual_wins,qual_average,rank,start_date,ties,wins
0,2017tur,5.0,5.0,0.0,5.0,5.0,1.0,0.0,5.0,1.0,0.0,0.0,0.0,0.0,2017-04-21 23:16:31.410000,frc3245,1.0,2017tur_qm111,ianbroadbooks@gmail.com,Ian Broadbooks,ianbroadbooks@gmail%2Ecom,2017-04-21 23:12:40,0,blue,"['frc16', 'frc5104']",15,0,15,15,90,60,qm,Houston,tur,USA,0.0,0.0,2017-04-22,TUR,0.0,29.752142,-95.357836,Turing Division,2017-04-19,TX,America/Chicago,3,Championship Division,8,0,0,1,0,True,111,2017-04-21 23:15:22,2017-04-21 23:12:01,1,1,1,1,1,0,1,1,1,0,True,387,1,4.0,0,83,0,27,297,120,150,2017-04-21 23:12:00,387,1,1,1,132,2017,3,1,4,3,400.0,2852.0,42.0,1.4,1400.0,1250.0,3.331024,105.534487,0,1,5,10,108.865511,0,0.0,31,2017-04-19,0,5
11,2017tur,3.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,2017-04-21 23:15:24.404000,frc1983,1.0,2017tur_qm111,kanoanakama@gmail.com,kanoa nakama,kanoanakama@gmail%2Ecom,2017-04-21 23:12:40,0,red,"['frc4270', 'frc1165']",0,0,0,15,75,60,qm,Houston,tur,USA,0.0,0.0,2017-04-22,TUR,0.0,29.752142,-95.357836,Turing Division,2017-04-19,TX,America/Chicago,3,Championship Division,8,0,0,0,0,False,111,2017-04-21 23:15:22,2017-04-21 23:12:01,1,1,1,1,1,0,1,1,0,0,False,255,1,0.0,0,0,0,0,180,80,100,2017-04-21 23:12:00,255,1,1,0,-132,2017,3,1,3,2,581.0,2612.0,17.0,1.3,1460.0,950.0,5.203962,70.344877,0,0,5,10,75.548839,0,0.0,40,2017-04-19,0,5
21,2017tur,3.0,0.0,0.0,0.0,3.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2017-04-21 23:15:19.997000,frc4270,1.0,2017tur_qm111,brian.titcomb27@gmail.com,Brian Titcomb,brian%2Etitcomb27@gmail%2Ecom,2017-04-21 23:12:40,0,red,"['frc1983', 'frc1165']",0,0,0,15,75,60,qm,Houston,tur,USA,0.0,0.0,2017-04-22,TUR,0.0,29.752142,-95.357836,Turing Division,2017-04-19,TX,America/Chicago,3,Championship Division,8,0,0,0,0,False,111,2017-04-21 23:15:22,2017-04-21 23:12:01,1,1,1,1,1,0,1,1,0,0,False,255,1,0.0,0,0,0,0,180,80,100,2017-04-21 23:12:00,255,1,1,0,-132,2017,3,1,3,2,518.0,2841.0,26.0,1.3,1520.0,1100.0,-59.389223,153.802896,0,0,6,10,94.413673,0,0.0,36,2017-04-19,0,4
31,2017tur,5.0,0.0,0.0,0.0,4.0,1.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,2017-04-21 23:07:38.961000,frc6554,1.0,2017tur_qm110,ianbroadbooks@gmail.com,Ian Broadbooks,ianbroadbooks@gmail%2Ecom,2017-04-21 23:04:20,0,blue,"['frc3219', 'frc3737']",0,0,0,15,75,60,qm,Houston,tur,USA,0.0,0.0,2017-04-22,TUR,0.0,29.752142,-95.357836,Turing Division,2017-04-19,TX,America/Chicago,3,Championship Division,8,0,0,0,0,False,11

### Other groupings

In [36]:
combined_df_by_team_event = combined_df[combined_df.match_found == 1].groupby(
                                    by=['team_key', 'event_key']).agg('mean')

combined_df_by_team_event.reset_index(inplace=True)
combined_df_by_team_event.sort_values(['team_key', 'event_week'], ascending=[True, False], inplace=True)

print(len(combined_df_by_team_event))
combined_df_by_team_event.head()

189


,team_key,event_key,rating_overall_gear_efficiency,rating_overall_gear_efficiency_auto,rating_overall_gear_placement_auto,rating_overall_pilot_competency,rating_overall_robot_stability,rating_scoring_airship_climb,rating_scoring_base_line_made_auto,rating_scoring_gears_made,rating_scoring_gears_made_auto,rating_scoring_high_goals_made,rating_scoring_high_goals_made_auto,rating_scoring_low_goals_made,rating_scoring_low_goals_made_auto,match_found,adjustPoints,autoFuelHigh,autoFuelLow,autoFuelPoints,autoMobilityPoints,autoPoints,autoRotorPoints,event_district_display_name,event_district_key,event_first_event_id,event_lat,event_lng,event_type,event_week,foulCount,foulPoints,is_winner,kPaBonusPoints,kPaRankingPointAchieved,match_number,robot1Auto,robot2Auto,robot3Auto,rotor1Auto,rotor1Engaged,rotor2Auto,rotor2Engaged,rotor3Engaged,rotor4Engaged,rotorBonusPoints,rotorRankingPointAchieved,score,set_number,tba_rpEarned,techFoulCount,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,totalPoints,touchpadFar,touchpadMiddle,touchpadNear,winning_margin,year,robotAutoCount,rotorAutoCount,rotorEngagedCount,takeoffCount,Auto,Match Points,Pressure,Ranking Score,Rotor,Touchpad,ccwms,dprs,is_finals_team,is_playoff_team,losses,matches_played,oprs,post_qual_wins,qual_average,rank,ties,wins
0,frc100,2017cada,1.222222,0.000000,0.000000,2.222222,3.222222,0.777778,0.666667,0.777778,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,2.333333,0.777778,2.555556,13.888889,49.777778,33.333333,0.0,0.0,22521.0,38.541889,-121.759586,0.0,3.0,0.222222,1.111111,0.444444,0.0,0.111111,40.555556,1.000000,0.888889,0.888889,0.555556,1.0,0.0,1.0,0.111111,0.000000,0.0,0.000000,200.777778,1.0,1.000000,0.666667,13.222222,0.333333,4.333333,149.888889,62.222222,83.333333,200.777778,0.666667,0.333333,0.666667,-11.333333,2017.0,2.777778,0.555556,2.111111,1.666667,388.0,1787.0,62.0,0.77,840.0,750.0,-6.638987,71.136739,0.0,0.0,6.0,9.0,64.497752,0.0,0.0,44.0,0.0,3.0
1,frc1056,2017cada,3.000000,0.900000,0.200000,3.000000,3.500000,0.700000,0.000000,1.700000,0.200000,0.000000,5.300000,0.0,0.0,1.0,0.0,9.400000,0.000000,9.400000,14.500000,65.900000,42.000000,0.0,0.0,22521.0,38.541889,-121.759586,0.0,3.0,0.200000,1.500000,0.700000,0.0,0.100000,51.300000,1.000000,1.000000,0.900000,0.700000,1.0,0.0,0.9,0.300000,0.000000,0.0,0.000000,241.500000,1.0,1.500000,0.300000,12.300000,0.700000,4.100000,174.100000,60.000000,110.000000,241.500000,0.900000,0.700000,0.600000,55.300000,2017.0,2.900000,0.700000,2.200000,2.200000,506.0,2067.0,122.0,1.44,860.0,950.0,11.418404,63.450057,0.0,1.0,3.0,9.0,74.868461,1.0,0.0,8.0,0.0,6.0
2,frc1072,2017cada,3.444444,1.000000,0.111111,1.888889,3.777778,0.777778,0.222222,2.111111,0.111111,1.111111,1.666667,0.0,0.0,1.0,0.0,2.222222,0.000000,2.222222,13.333333,35.555556,20.000000,0.0,0.0,22521.0,38.541889,-121.759586,0.0,3.0,0.111111,5.555556,0.555556,0.0,0.000000,43.222222,0.888889,1.000000,0.777778,0.333333,1.0,0.0,1.0,0.555556,0.000000,0.0,0.000000,214.222222,1.0,1.111111,0.777778,3.000000,0.000000,0.888889,173.111111,88.888889,83.333333,214.222222,0.666667,0.444444,0.555556,-12.111111,2017.0,2.666667,0.333333,2.555556,1.666667,320.0,1928.0,28.0,1.11,980.0,750.0,-14.813528,78.854740,0.0,1.0,4.0,9.0,64.041212,2.0,0.0,27.0,0.0,5.0
3,frc108,2017tur,4.181818,1.636364,0.272727,3.909091,4.272727,0.909091,0.181818,3.000000,0.363636,0.090909,0.000000,0.0,0.0,1.0,0.0,5.454545,0.000000,5.454545,14.545455,36.363636,16.363636,0.0,0.0,0.0,29.752142,-95.357836,3.0,8.0,0.090909,43.636364,0.363636,0.0,0.000000,59.181818,1.000000,0.909091,1.000000,0.272727,1.0,0.0,1.0,1.000000,0.727273,0.0,0.727273,332.272727,1.0,1.454545,0.000000,15.272727,0.000000,5.000000,252.272727,138.181818,109.090909,332.272727,0.909091,0.909091,0.363636,18.090909,2017.0,2.909091,0.272727,3.727273,2.181818,460.0,3086.0,26.0,1.50,1500.0,1100.0,38.493471,87.810154,0.0,1.0,5.0,10.0,126.303625,0.0,0.0,23.0,0.0,5.0
4,frc1165,2017tur,1.750000,0.000000,0.000000,1.000000,3.

In [32]:
combined_df_by_team = combined_df_by_team_event.groupby(by=['team_key']).agg('mean')

combined_df_by_team.reset_index(inplace=True)
combined_df_by_team.sort_values('team_key', ascending=True, inplace=True)

print(len(combined_df_by_team))
combined_df_by_team.head()

176


,team_key,rating_overall_gear_efficiency,rating_overall_gear_efficiency_auto,rating_overall_gear_placement_auto,rating_overall_pilot_competency,rating_overall_robot_stability,rating_scoring_airship_climb,rating_scoring_base_line_made_auto,rating_scoring_gears_made,rating_scoring_gears_made_auto,rating_scoring_high_goals_made,rating_scoring_high_goals_made_auto,rating_scoring_low_goals_made,rating_scoring_low_goals_made_auto,match_found,adjustPoints,autoFuelHigh,autoFuelLow,autoFuelPoints,autoMobilityPoints,autoPoints,autoRotorPoints,event_district_display_name,event_district_key,event_first_event_id,event_lat,event_lng,event_type,event_week,foulCount,foulPoints,is_winner,kPaBonusPoints,kPaRankingPointAchieved,match_number,robot1Auto,robot2Auto,robot3Auto,rotor1Auto,rotor1Engaged,rotor2Auto,rotor2Engaged,rotor3Engaged,rotor4Engaged,rotorBonusPoints,rotorRankingPointAchieved,score,set_number,tba_rpEarned,techFoulCount,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,totalPoints,touchpadFar,touchpadMiddle,touchpadNear,winning_margin,year,robotAutoCount,rotorAutoCount,rotorEngagedCount,takeoffCount,Auto,Match Points,Pressure,Ranking Score,Rotor,Touchpad,ccwms,dprs,is_finals_team,is_playoff_team,losses,matches_played,oprs,post_qual_wins,qual_average,rank,ties,wins
0,frc100,1.222222,0.000000,0.000000,2.222222,3.222222,0.777778,0.666667,0.777778,0.000000,0.000000,0.000000,0.0,0.0,1.0,0.0,2.333333,0.777778,2.555556,13.888889,49.777778,33.333333,0.0,0.0,22521.0,38.541889,-121.759586,0.0,3.0,0.222222,1.111111,0.444444,0.0,0.111111,40.555556,1.000000,0.888889,0.888889,0.555556,1.0,0.0,1.0,0.111111,0.000000,0.0,0.000000,200.777778,1.0,1.000000,0.666667,13.222222,0.333333,4.333333,149.888889,62.222222,83.333333,200.777778,0.666667,0.333333,0.666667,-11.333333,2017.0,2.777778,0.555556,2.111111,1.666667,388.0,1787.0,62.0,0.77,840.0,750.0,-6.638987,71.136739,0.0,0.0,6.0,9.0,64.497752,0.0,0.0,44.0,0.0,3.0
1,frc1056,3.000000,0.900000,0.200000,3.000000,3.500000,0.700000,0.000000,1.700000,0.200000,0.000000,5.300000,0.0,0.0,1.0,0.0,9.400000,0.000000,9.400000,14.500000,65.900000,42.000000,0.0,0.0,22521.0,38.541889,-121.759586,0.0,3.0,0.200000,1.500000,0.700000,0.0,0.100000,51.300000,1.000000,1.000000,0.900000,0.700000,1.0,0.0,0.9,0.300000,0.000000,0.0,0.000000,241.500000,1.0,1.500000,0.300000,12.300000,0.700000,4.100000,174.100000,60.000000,110.000000,241.500000,0.900000,0.700000,0.600000,55.300000,2017.0,2.900000,0.700000,2.200000,2.200000,506.0,2067.0,122.0,1.44,860.0,950.0,11.418404,63.450057,0.0,1.0,3.0,9.0,74.868461,1.0,0.0,8.0,0.0,6.0
2,frc1072,3.444444,1.000000,0.111111,1.888889,3.777778,0.777778,0.222222,2.111111,0.111111,1.111111,1.666667,0.0,0.0,1.0,0.0,2.222222,0.000000,2.222222,13.333333,35.555556,20.000000,0.0,0.0,22521.0,38.541889,-121.759586,0.0,3.0,0.111111,5.555556,0.555556,0.0,0.000000,43.222222,0.888889,1.000000,0.777778,0.333333,1.0,0.0,1.0,0.555556,0.000000,0.0,0.000000,214.222222,1.0,1.111111,0.777778,3.000000,0.000000,0.888889,173.111111,88.888889,83.333333,214.222222,0.666667,0.444444,0.555556,-12.111111,2017.0,2.666667,0.333333,2.555556,1.666667,320.0,1928.0,28.0,1.11,980.0,750.0,-14.813528,78.854740,0.0,1.0,4.0,9.0,64.041212,2.0,0.0,27.0,0.0,5.0
3,frc108,4.181818,1.636364,0.272727,3.909091,4.272727,0.909091,0.181818,3.000000,0.363636,0.090909,0.000000,0.0,0.0,1.0,0.0,5.454545,0.000000,5.454545,14.545455,36.363636,16.363636,0.0,0.0,0.0,29.752142,-95.357836,3.0,8.0,0.090909,43.636364,0.363636,0.0,0.000000,59.181818,1.000000,0.909091,1.000000,0.272727,1.0,0.0,1.0,1.000000,0.727273,0.0,0.727273,332.272727,1.0,1.454545,0.000000,15.272727,0.000000,5.000000,252.272727,138.181818,109.090909,332.272727,0.909091,0.909091,0.363636,18.090909,2017.0,2.909091,0.272727,3.727273,2.181818,460.0,3086.0,26.0,1.50,1500.0,1100.0,38.493471,87.810154,0.0,1.0,5.0,10.0,126.303625,0.0,0.0,23.0,0.0,5.0
4,frc1165,1.750000,0.000000,0.000000,1.000000,3.250000,1.000000,0.000000,1.500000,0.250000,0.000000,0

### Examples from various groupings

In [33]:
combined_df_by_team[combined_df_by_team.team_key == 'frc2102']

,team_key,rating_overall_gear_efficiency,rating_overall_gear_efficiency_auto,rating_overall_gear_placement_auto,rating_overall_pilot_competency,rating_overall_robot_stability,rating_scoring_airship_climb,rating_scoring_base_line_made_auto,rating_scoring_gears_made,rating_scoring_gears_made_auto,rating_scoring_high_goals_made,rating_scoring_high_goals_made_auto,rating_scoring_low_goals_made,rating_scoring_low_goals_made_auto,match_found,adjustPoints,autoFuelHigh,autoFuelLow,autoFuelPoints,autoMobilityPoints,autoPoints,autoRotorPoints,event_district_display_name,event_district_key,event_first_event_id,event_lat,event_lng,event_type,event_week,foulCount,foulPoints,is_winner,kPaBonusPoints,kPaRankingPointAchieved,match_number,robot1Auto,robot2Auto,robot3Auto,rotor1Auto,rotor1Engaged,rotor2Auto,rotor2Engaged,rotor3Engaged,rotor4Engaged,rotorBonusPoints,rotorRankingPointAchieved,score,set_number,tba_rpEarned,techFoulCount,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,totalPoints,touchpadFar,touchpadMiddle,touchpadNear,winning_margin,year,robotAutoCount,rotorAutoCount,rotorEngagedCount,takeoffCount,Auto,Match Points,Pressure,Ranking Score,Rotor,Touchpad,ccwms,dprs,is_finals_team,is_playoff_team,losses,matches_played,oprs,post_qual_wins,qual_average,rank,ties,wins
32,frc2102,3.066667,1.87619,0.142857,2.857143,3.561905,0.419048,0.142857,1.742857,0.27619,6.0,6.952381,0.0,0.0,1.0,0.0,5.504762,0.0,5.504762,12.809524,47.457143,29.142857,0.0,0.0,15012.0,33.756424,-111.458593,1.0,4.0,0.228571,7.380952,0.409524,0.0,0.047619,39.885714,0.933333,0.761905,0.866667,0.485714,1.0,0.0,1.0,0.485714,0.0,0.0,0.0,210.838095,1.0,0.866667,0.304762,11.971429,11.580952,5.047619,156.0,80.0,70.952381,210.838095,0.447619,0.32381,0.647619,-21.066667,2017.0,2.561905,0.485714,2.485714,1.419048,427.666667,2012.666667,79.333333,1.143333,1020.0,716.666667,-1.983183,62.545241,0.333333,0.666667,4.333333,9.0,60.562058,2.333333,0.0,34.0,0.0,4.666667


In [34]:
combined_df_by_team_event[combined_df_by_team_event.team_key == 'frc2102']

,team_key,event_key,rating_overall_gear_efficiency,rating_overall_gear_efficiency_auto,rating_overall_gear_placement_auto,rating_overall_pilot_competency,rating_overall_robot_stability,rating_scoring_airship_climb,rating_scoring_base_line_made_auto,rating_scoring_gears_made,rating_scoring_gears_made_auto,rating_scoring_high_goals_made,rating_scoring_high_goals_made_auto,rating_scoring_low_goals_made,rating_scoring_low_goals_made_auto,match_found,adjustPoints,autoFuelHigh,autoFuelLow,autoFuelPoints,autoMobilityPoints,autoPoints,autoRotorPoints,event_district_display_name,event_district_key,event_first_event_id,event_lat,event_lng,event_type,event_week,foulCount,foulPoints,is_winner,kPaBonusPoints,kPaRankingPointAchieved,match_number,robot1Auto,robot2Auto,robot3Auto,rotor1Auto,rotor1Engaged,rotor2Auto,rotor2Engaged,rotor3Engaged,rotor4Engaged,rotorBonusPoints,rotorRankingPointAchieved,score,set_number,tba_rpEarned,techFoulCount,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,totalPoints,touchpadFar,touchpadMiddle,touchpadNear,winning_margin,year,robotAutoCount,rotorAutoCount,rotorEngagedCount,takeoffCount,Auto,Match Points,Pressure,Ranking Score,Rotor,Touchpad,ccwms,dprs,is_finals_team,is_playoff_team,losses,matches_played,oprs,post_qual_wins,qual_average,rank,ties,wins
35,frc2102,2017tur,3.714286,3.285714,0.428571,4.285714,4.428571,0.571429,0.285714,1.857143,0.428571,5.714286,11.142857,0.0,0.0,1.0,0.0,9.285714,0.0,9.285714,13.571429,57.142857,34.285714,0.0,0.0,0.0,29.752142,-95.357836,3.0,8.0,0.285714,2.857143,0.142857,0.0,0.142857,42.714286,1.0,0.714286,1.0,0.571429,1.0,0.0,1.0,0.857143,0.0,0.0,0.0,241.714286,1.0,0.428571,0.714286,14.428571,0.142857,4.571429,181.714286,91.428571,85.714286,241.714286,0.857143,0.285714,0.571429,-57.285714,2017.0,2.714286,0.571429,2.857143,1.714286,584.0,2641.0,116.0,0.80,1320.0,1000.0,-17.386072,90.605959,0.0,0.0,7.0,10.0,73.219887,0.0,0.0,59.0,0.0,3.0
33,frc2102,2017cada,2.285714,0.142857,0.000000,4.285714,2.857143,0.285714,0.142857,1.571429,0.000000,3.285714,5.714286,0.0,0.0,1.0,0.0,5.428571,0.0,5.428571,12.857143,35.428571,17.142857,0.0,0.0,22521.0,38.541889,-121.759586,0.0,3.0,0.000000,9.285714,0.285714,0.0,0.000000,41.142857,1.0,0.571429,1.0,0.285714,1.0,0.0,1.0,0.000000,0.0,0.0,0.0,174.000000,1.0,0.571429,0.000000,11.285714,0.000000,3.571429,129.285714,68.571429,57.142857,174.000000,0.285714,0.285714,0.571429,-50.714286,2017.0,2.571429,0.285714,2.000000,1.142857,343.0,1675.0,75.0,0.88,820.0,600.0,-22.039435,63.147163,0.0,1.0,5.0,9.0,41.107728,1.0,0.0,40.0,0.0,4.0
34,frc2102,2017casd,3.200000,2.200000,0.000000,0.000000,3.400000,0.400000,0.000000,1.800000,0.400000,9.000000,4.000000,0.0,0.0,1.0,0.0,1.800000,0.0,1.800000,12.000000,49.800000,36.000000,0.0,0.0,22515.0,32.975240,-117.258357,0.0,1.0,0.400000,10.000000,0.800000,0.0,0.000000,35.800000,0.8,1.000000,0.6,0.600000,1.0,0.0,1.0,0.600000,0.0,0.0,0.0,216.800000,1.0,1.600000,0.200000,10.200000,34.600000,7.000000,157.000000,80.000000,70.000000,216.800000,0.200000,0.400000,0.800000,44.800000,2017.0,2.400000,0.600000,2.600000,1.400000,356.0,1722.0,47.0,1.75,920.0,550.0,33.475959,33.882600,1.0,1.0,1.0,8.0,67.358559,6.0,0.0,3.0,0.0,7.0


In [35]:
combined_df[combined_df.team_key == 'frc2102']

,event_key,rating_overall_gear_efficiency,rating_overall_gear_efficiency_auto,rating_overall_gear_placement_auto,rating_overall_pilot_competency,rating_overall_robot_stability,rating_scoring_airship_climb,rating_scoring_base_line_made_auto,rating_scoring_gears_made,rating_scoring_gears_made_auto,rating_scoring_high_goals_made,rating_scoring_high_goals_made_auto,rating_scoring_low_goals_made,rating_scoring_low_goals_made_auto,scored_at,team_key,match_found,match_key,scored_by_email,scored_by_name,scored_by_user_key,actual_time,adjustPoints,alliance,alliance_team_keys,autoFuelHigh,autoFuelLow,autoFuelPoints,autoMobilityPoints,autoPoints,autoRotorPoints,comp_level,event_city,event_code,event_country,event_district_display_name,event_district_key,event_end_date,event_first_event_code,event_first_event_id,event_lat,event_lng,event_name,event_start_date,event_state_prov,event_timezone,event_type,event_type_string,event_week,foulCount,foulPoints,is_winner,kPaBonusPoints,kPaRankingPointAchieved,match_number,post_result_time,predicted_time,robot1Auto,robot2Auto,robot3Auto,rotor1Auto,rotor1Engaged,rotor2Auto,rotor2Engaged,rotor3Engaged,rotor4Engaged,rotorBonusPoints,rotorRankingPointAchieved,score,set_number,tba_rpEarned,techFoulCount,teleopFuelHigh,teleopFuelLow,teleopFuelPoints,teleopPoints,teleopRotorPoints,teleopTakeoffPoints,time,totalPoints,touchpadFar,touchpadMiddle,touchpadNear,winning_margin,year,robotAutoCount,rotorAutoCount,rotorEngagedCount,takeoffCount,Auto,Match Points,Pressure,Ranking Score,Rotor,Touchpad,ccwms,dprs,is_finals_team,is_playoff_team,losses,matches_played,oprs,post_qual_wins,qual_average,rank,start_date,ties,wins
552,2017tur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,frc2102,0.0,2017tur_qm112,0,0,0,2017-04-21 23:22:00,0,red,"['frc108', 'frc399']",3,0,3,15,18,0,qm,Houston,tur,USA,0.0,0.0,2017-04-22,TUR,0.0,29.752142,-95.357836,Turing Division,2017-04-19,TX,America/Chicago,3,Championship Division,8,0,35,1,0,False,112,2017-04-21 23:24:42,2017-04-21 23:20:11,1,1,1,0,1,0,1,1,1,0,True,363,1,3.0,0,1,0,0,310,160,150,2017-04-21 23:20:00,363,1,1,1,108,2017,3,0,4,3,584.0,2641.0,116.0,0.80,1320.0,1000.0,-17.386072,90.605959,0,0,7,10,73.219887,0,0.0,59,2017-04-19,0,3
545,2017tur,5.0,5.0,1.0,5.0,5.0,1.0,0.0,3.0,1.0,0.0,10.0,0.0,0.0,2017-04-21 21:12:28.180000,frc2102,1.0,2017tur_qm96,dflores0003@gmail.com,Darian Flores,dflores0003@gmail%2Ecom,2017-04-21 21:09:48,0,blue,"['frc1165', 'frc3794']",8,0,8,15,83,60,qm,Houston,tur,USA,0.0,0.0,2017-04-22,TUR,0.0,29.752142,-95.357836,Turing Division,2017-04-19,TX,America/Chicago,3,Championship Division,8,1,0,0,0,False,96,2017-04-21 21:12:41,2017-04-21 21:09:01,1,1,1,1,1,0,1,1,0,0,False,263,1,0.0,0,2,0,0,180,80,100,2017-04-21 21:12:00,263,1,1,0,-83,2017,3,1,3,2,584.0,2641.0,116.0,0.80,1320.0,1000.0,-17.386072,90.605959,0,0,7,10,73.219887,0,0.0,59,2017-04-19,0,3
553,2017tur,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,frc2102,0.0,2017tur_qm88,0,0,0,2017-04-21 20:07:05,0,blue,"['frc5829', 'frc6390']",9,0,9,15,84,60,qm,Houston,tur,USA,0.0,0.0,2017-04-22,TUR,0.0,29.752142,-95.357836,Turing Division,2017-04-19,TX,America/Chicago,3,Championship Division,8,0,0,0,0,False,88,2017-04-21 20:12:14,2017-04-21 20:08:02,1,1,1,1,1,0,1,1,0,0,False,264,1,0.0,0,0,0,0,180,80,100,2017-04-21 20:08:00,264,0,1,1,-41,2017,3,1,3,2,584.0,2641.0,116.0,0.80,1320.0,1000.0,-17.386072,90.605959,0,0,7,10,73.219887,0,0.0,59,2017-04-19,0,3
546,2017tur,5.0,5.0,1.0,5.0,5.0,1.0,0.0,2.0,1.0,0.0,5.0,0.0,0.0,2017-04-21 15:33:17.614000,frc2102,1.0,2017tur_qm68,brian.titcomb27@gmail.com,Brian Titcomb,brian%2Etitcomb27@gmail%2Ecom,2017-04-21 15:30:33,0,red,"['frc4610', 'frc6225']",5,0,5,15,80,60,qm,Houston,tur,USA,0.0,0.0,2017-04-22,TUR,0.0,29.752142,-95.357836,Turing Division,2017-04-19,TX,America/Chicago,3,Championship Division,8,0,0,0,0,False,68,2017-04-21 15:33:19,2017-04-21 15:32:10,1,1,1,1,1,0,1,0,0,0,False,220,1,0.0,2,0,0,0,140,40,100,2017-04-21 15:32:00,220,1,0,1,-65,2017,3,1,2,2,584.0,2641.0,116.0,0